In [ ]:
import os
import json
import gradio as gr
from dotenv import load_dotenv
from openai import OpenAI

In [ ]:
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

In [ ]:
openai = OpenAI()
MODEL = 'gpt-4o-mini'

In [ ]:
system_message = "You are a helpful assistant for a Web Development Company called Abh-devs. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [ ]:
# Employees list
employees = {
    "Alice Johnson": "CEO",
    "Bob Smith": "CTO",
    "Clara Martinez": "Frontend Developer",
    "David Lee": "Backend Developer",
    "Eva Brown": "Fullstack Developer",
    "Frank Wilson": "UI/UX Designer",
    "Grace Kim": "Project Manager",
    "Henry Clark": "QA Engineer",
    "Isabella Davis": "DevOps Engineer",
}

In [ ]:
# Tool: Get all employees
def get_all_employees():
    print("Tool get_all_employees called")
    return [{"name": name, "role": role} for name, role in employees.items()]

all_employees_function = {
    "name": "get_all_employees",
    "description": "Get the list of all employees and their roles.",
    "parameters": {
        "type": "object",
        "properties": {},
        "additionalProperties": False
    }
}

In [ ]:
# Tool: Get role by employee name
def get_employee_role(employee_name):
    print(f"Tool get_employee_role called for {employee_name}")
    return employees.get(employee_name, "Unknown employee")

employee_role_function = {
    "name": "get_employee_role",
    "description": "Get the role of an employee by their name.",
    "parameters": {
        "type": "object",
        "properties": {
            "employee_name": {
                "type": "string",
                "description": "The full name of the employee."
            },
        },
        "required": ["employee_name"],
        "additionalProperties": False
    }
}

In [ ]:
# Register Tools
tools = [
    {"type": "function", "function": all_employees_function},
    {"type": "function", "function": employee_role_function},
]

In [ ]:
# Handle Tool Calls
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)

    if tool_call.function.name == "get_all_employees":
        result = get_all_employees()
        response = {"role": "tool", "content": json.dumps(result), "tool_call_id": tool_call.id}
        return response, None

    elif tool_call.function.name == "get_employee_role":
        employee_name = arguments.get("employee_name")
        role = get_employee_role(employee_name)
        response = {"role": "tool", "content": json.dumps({"employee_name": employee_name, "role": role}), "tool_call_id": tool_call.id}
        return response, employee_name

In [ ]:
# Chat Function
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        response, _ = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)

    return response.choices[0].message.content


In [ ]:
# Gradio Chat Interface
gr.ChatInterface(fn=chat, type="messages", title="Web Dev Company Assistant").launch()